In [6]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import geojson
import shapely
from shapely.geometry import Polygon, Point, mapping
import rasterio
from rasterio import plot, mask
from rasterio.plot import show
import zipfile
import time
import io
from PIL import Image
from PIL.TiffTags import TAGS
from osgeo import gdal

OUTDIR = "example_outputs"   # or keep your original name
os.makedirs(OUTDIR, exist_ok=True)

In [8]:
### BORDER 2 (2025, San Diego County)

# Name and IRWIN ID
INCIDENT_NAME = "BORDER 2"
IRWIN_ID = "386ebecf-af18-442d-b5d9-e2772b91914a"


# Ignition alert and containment timestamps
IGNITION_TIME = "2025-01-23T13:53:00"
CONTAINMENT_TIME = "2025-01-28T18:00:00"


# Ignition coordinates
IGNITION_LAT = 32.601
IGNITION_LON = -116.841
IGNITION_POINT = Point((IGNITION_LON, IGNITION_LAT))


# Write incident ignition data to GeoJSON
ignition_feature = geojson.Feature(
    geometry=mapping(IGNITION_POINT),
    properties={
        "incident_name": INCIDENT_NAME,
        "irwin_id": IRWIN_ID,
        "ignition_time": IGNITION_TIME,
        "containment_time": CONTAINMENT_TIME,
    }
)


# FeatureCollection
feature_collection = geojson.FeatureCollection([ignition_feature])

# Write to file
with open("example_outputs/border2_ignition.geojson", "w") as f:
    geojson.dump(feature_collection, f, indent=2)

In [9]:
# BORDER 2 - Retrieval via Firemap/Pylaski API (up to 2 years back)
timedelta = pd.Timedelta(hours=24)


# Read t=0 data from ignition file
ignition_filepath = "example_outputs/border2_ignition.geojson"
with open(ignition_filepath) as f:
    gj = json.load(f)


# Define time constraints for t=0
ignition_json = gj['features'][0]
start_time = ignition_json["properties"]["ignition_time"]  # Ignition time
perimeter_update_time = datetime.fromisoformat(start_time) + timedelta
end_time = perimeter_update_time.strftime("%Y-%m-%dT%H:%M:%S")  # 24 hours after ignition time


# Read ignition coordinate point
coordinate = ignition_json["geometry"]["coordinates"]
lon, lat = coordinate[0], coordinate[1]

# GET to nearest weather station for wX observables
FIREMAP_WX_URL = "https://firemap.sdsc.edu/pylaski/stations/data?"
timestamp = int(time.time() * 1000)
FIREMAP_WX_PARAMS = {
        'selection': 'closestTo',
        'lat': str(lat),
        'lon': str(lon),
        'observable': ['wind_speed', 'wind_direction'],
        'from': start_time,
        'to': end_time,
        'callback': 'wxData',
        '_': str(timestamp)
    }
FIREMAP_WX_RESPONSE = requests.get(FIREMAP_WX_URL, params=FIREMAP_WX_PARAMS)
print(FIREMAP_WX_RESPONSE.url)


# Save wX observables to JSON
WX_TEXT = FIREMAP_WX_RESPONSE.text.strip()


# Remove callback wrapper if necessary
if WX_TEXT.startswith('wxData(') and WX_TEXT.endswith(')'):
    WX_JSON = WX_TEXT[len('wxData('):-1]
    WX_OBS = json.loads(WX_JSON)
else:
    WX_OBS = FIREMAP_WX_RESPONSE.json()


# Extract wind speed and wind direction observed values
WIND_SPEED_LIST = WX_OBS["features"][0]["properties"]["wind_speed"]
WIND_DIRECTION_LIST = WX_OBS["features"][0]["properties"]["wind_direction"]


# Add wX observations to ignition json
ignition_json["properties"]["wind_speed_list"] = WIND_SPEED_LIST
ignition_json["properties"]["wind_direction_list"] = WIND_DIRECTION_LIST
ignition_json["properties"]["wX_end_time"] = end_time


# Write updated ignition json to ignition file
with open(ignition_filepath, "w") as f:
    json.dump(ignition_json, f, indent=2)

https://firemap.sdsc.edu/pylaski/stations/data?selection=closestTo&lat=32.601&lon=-116.841&observable=wind_speed&observable=wind_direction&from=2025-01-23T13%3A53%3A00&to=2025-01-24T13%3A53%3A00&callback=wxData&_=1770670360278


In [10]:
ignition_json

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-116.841, 32.601]},
 'properties': {'incident_name': 'BORDER 2',
  'irwin_id': '386ebecf-af18-442d-b5d9-e2772b91914a',
  'ignition_time': '2025-01-23T13:53:00',
  'containment_time': '2025-01-28T18:00:00',
  'wind_speed_list': [4.47,
   15.2,
   11.62,
   9.84,
   11.17,
   9.84,
   9.84,
   7.6,
   8.94,
   12.52,
   12.52,
   12.52,
   11.62,
   11.62,
   9.84,
   11.62,
   10.28,
   11.17,
   13.86,
   13.86,
   10.28,
   5.81,
   6.26,
   6.26],
  'wind_direction_list': [57.0,
   80.0,
   84.0,
   85.0,
   84.0,
   81.0,
   82.0,
   77.0,
   78.0,
   84.0,
   79.0,
   81.0,
   86.0,
   81.0,
   88.0,
   93.0,
   106.0,
   107.0,
   90.0,
   90.0,
   90.0,
   88.0,
   102.0,
   114.0],
  'wX_end_time': '2025-01-24T13:53:00'}}